In [2]:
import numpy as np
from sklearn.linear_model import LinearRegression
import torch
import torch.optim as optim
import torch.nn as nn
from torchviz import make_dot
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

In [3]:
%run -i data_generation/simple_linear_regression.py
%run -i model_configuration/v0.py
%run -i data_preparation/v0.py


In [4]:
true_b = 1
true_w = 2
N = 100
np.random.seed(42)
x = np.random.rand(N, 1)
print(x.shape)
epsilon = (-0.1 * np.random.randn(N, 1))
y = true_w * x + true_b + epsilon

(100, 1)


In [5]:
idx = np.arange(N)
np.random.shuffle(idx)
train_idx = idx[:int(0.8 * N)]
val_idx = idx[int(0.8 * N):]
x_train, y_train = x[train_idx], y[train_idx]
x_val, y_val = x[val_idx],y[val_idx]

In [6]:
device = 'gpu' if torch.cuda.is_available() else 'cpu'

In [7]:
x_train_tensor = torch.as_tensor(x_train).float().to(device)
y_train_tensor = torch.as_tensor(y_train).float().to(device)
print(type(x_train),type(x_train_tensor), x_train_tensor.type())

<class 'numpy.ndarray'> <class 'torch.Tensor'> torch.FloatTensor


In [1]:
# %model_training/v0.py

In [8]:
torch.manual_seed(42)
model = nn.Sequential(nn.Linear(1, 1)).to(device)
model.state_dict()

OrderedDict([('0.weight', tensor([[0.7645]])), ('0.bias', tensor([0.8300]))])

In [9]:

lr = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr = lr)
loss_fn = nn.MSELoss(reduction='mean')
epochs = 1000

In [10]:
epochs = 1000
for _ in range(epochs):
    model.train()
    predictions = model.forward(x_train_tensor)
    loss = loss_fn(predictions, y_train_tensor)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
print(model.state_dict())

OrderedDict([('0.weight', tensor([[2.0310]])), ('0.bias', tensor([0.9765]))])


In [13]:
%%writefile model_configuration/v1.py
def make_train_step_fn(model, loss_fn, optimizer):
    def perform_train_step_fn(x, y):
        model.train()
        yhat = model(x)
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()
    return perform_train_step_fn
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = nn.Linear(1, 1).to(device)
lr = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr = lr)
loss_fn = nn.MSELoss(reduction='mean')
performer_train_step = make_train_step_fn(model, loss_fn, optimizer)
performer_train_step


Overwriting model_configuration/v1.py


In [12]:
%run -i model_configuration/v1.py

In [15]:
losses = []
epochs = 1000
for _ in range(epochs):
    losss = performer_train_step(x_train_tensor, y_train_tensor)
    losses.append(loss)
print(losses)
print(model.state_dict())

[tensor(0.0080, grad_fn=<MseLossBackward0>), tensor(0.0080, grad_fn=<MseLossBackward0>), tensor(0.0080, grad_fn=<MseLossBackward0>), tensor(0.0080, grad_fn=<MseLossBackward0>), tensor(0.0080, grad_fn=<MseLossBackward0>), tensor(0.0080, grad_fn=<MseLossBackward0>), tensor(0.0080, grad_fn=<MseLossBackward0>), tensor(0.0080, grad_fn=<MseLossBackward0>), tensor(0.0080, grad_fn=<MseLossBackward0>), tensor(0.0080, grad_fn=<MseLossBackward0>), tensor(0.0080, grad_fn=<MseLossBackward0>), tensor(0.0080, grad_fn=<MseLossBackward0>), tensor(0.0080, grad_fn=<MseLossBackward0>), tensor(0.0080, grad_fn=<MseLossBackward0>), tensor(0.0080, grad_fn=<MseLossBackward0>), tensor(0.0080, grad_fn=<MseLossBackward0>), tensor(0.0080, grad_fn=<MseLossBackward0>), tensor(0.0080, grad_fn=<MseLossBackward0>), tensor(0.0080, grad_fn=<MseLossBackward0>), tensor(0.0080, grad_fn=<MseLossBackward0>), tensor(0.0080, grad_fn=<MseLossBackward0>), tensor(0.0080, grad_fn=<MseLossBackward0>), tensor(0.0080, grad_fn=<MseLoss

In [18]:
class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __getitem__(self, index):
        return(self.x[index], self.y[index])
    def __len__(self):
        return len(self.x)

In [21]:
x_train_tensor = torch.as_tensor(x_train).float().to(device)
y_train_tensor = torch.as_tensor(y_train).float().to(device)
train_data = CustomDataset(x_train_tensor, y_train_tensor)
print(train_data[0])

(tensor([0.7713]), tensor([2.6105]))
